In [1]:
from pprint import pprint
from typing import NamedTuple, Optional

import operator
import toolz


class Data(NamedTuple):
  name: str
  size: Optional[float]
  state: Optional[bool]
  date: Optional[str]
  time: Optional[str]


def generate_data():
  yield Data("3D Objects", 35.1, True, "06/06/2020", "00:55")
  yield Data(".pylint.d", 28.85, False, "03/17/2021", "23:46")
  yield Data("Favorites", None, None, None, None)
  yield Data("temp", 19.64, True, "05/14/2020", "22:39")
  yield Data(".gitconfig", 0.74, True, "02/28/2021", "22:55")
  yield Data(".bashrc", 4.12, True, "02/28/2021", "21:55")

回顾前面实现的格式化函数，最明显的重复体现在默认值的处理。因此先提取这个部分：


In [2]:
@toolz.curry
def with_default(f, default, value, need_default=lambda v: v is None):
  if need_default(value):
    return default
  return f(value)

去除默认值处理后，可以发现一些可以泛化的点：

- name格式化其实没有任何操作
- absolute size格式化其实就是调用了Python的format函数
- relative size和state的共同点是根据不同的条件判断，选取对应的值返回，本质是查表操作
- date time格式化其实就是执行了一个join操作

于是可以定义一些高度通用化的操作：


In [3]:
@toolz.curry
def by_formatter(formatter, value):
  return formatter.format(value)

In [4]:
@toolz.curry
def by_table(table, value):
  for callback, result in table.items():
    if callback(value):
      return result
  raise ValueError(f'cannot find a match in {table} for {value}')

In [5]:
@toolz.curry
def by_joiner(sep, segments):
  return sep.join(segments)

组合通用函数得到满足需求的格式化函数

In [6]:
format_name = operator.attrgetter('name')

format_abs_size = toolz.compose(
  with_default(
    by_formatter('{:.1f}MB'),
    '0.0MB'
  ),
  operator.attrgetter('size')
)

In [7]:
relative_size_table = {
  lambda v: v < 10: 'Small',
  lambda v: v >= 10 and v < 20: 'Medium',
  lambda v: v >= 20: 'Big'
}
format_relative_size = toolz.compose(
  with_default(
    by_table(relative_size_table),
    'Small'
  ),
  operator.attrgetter('size')
)

state_table = {
  toolz.identity: 'Success',
  toolz.complement(toolz.identity): 'Falure'
}
format_state = toolz.compose(
  with_default(
    by_table(state_table),
    'Success'
  ),
  operator.attrgetter('state')
)


In [8]:
format_date_time = toolz.compose(
  with_default(
    by_joiner(' '),
    'Unknown',
    need_default = toolz.compose(any, toolz.curried.map(lambda v: v is None))
  ),
  toolz.juxt(operator.attrgetter('date'), operator.attrgetter('time'))
)

In [9]:
juxt_a_row = toolz.juxt(
  format_name,
  format_abs_size,
  format_relative_size,
  format_state,
  format_date_time,
)

pprint(list(
  map(juxt_a_row, generate_data())
))

[('3D Objects', '35.1MB', 'Big', 'Success', '06/06/2020 00:55'),
 ('.pylint.d', '28.9MB', 'Big', 'Falure', '03/17/2021 23:46'),
 ('Favorites', '0.0MB', 'Small', 'Success', 'Unknown'),
 ('temp', '19.6MB', 'Medium', 'Success', '05/14/2020 22:39'),
 ('.gitconfig', '0.7MB', 'Small', 'Success', '02/28/2021 22:55'),
 ('.bashrc', '4.1MB', 'Small', 'Success', '02/28/2021 21:55')]
